In [ ]:
import os
import sys



from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Tuple
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from trading.environments.forex_env2_flat import ForexTradingEnv, Actions

class ModelTester:
    """Extended debugger for evaluating trained models on test data with VecEnv support."""
    
    def __init__(
        self, 
        model: PPO,
        vec_normalize: VecNormalize,
        test_df: pd.DataFrame,
        pair: str,
        save_path: Optional[Path] = None
    ):
        """
        Initialize model tester with vectorized environment support.
        
        Args:
            model: Trained PPO model
            vec_normalize: Saved VecNormalize wrapper with training statistics
            test_df: Test dataset
            pair: Currency pair being traded
            save_path: Path to save evaluation results
        """
        self.model = model
        self.test_df = test_df
        self.pair = pair
        self.save_path = Path(save_path) if save_path else Path("model_tests")
        self.save_path.mkdir(exist_ok=True)
        
        # Create vectorized environment for testing
        self.env = self._create_test_env(vec_normalize)
        
        # Initialize tracking containers
        self.trades = []
        self.balance_history = []
        self.position_history = []
        self.price_history = []
        self.reward_history = []
        self.pnl_history = []
        
    def _create_test_env(self, vec_normalize: VecNormalize) -> VecNormalize:
        """
        Create properly wrapped test environment with saved normalization statistics.
        
        Args:
            vec_normalize: VecNormalize wrapper from training with saved statistics
            
        Returns:
            Properly configured test environment
        """
        # Create base environment
        def make_env():
            env = ForexTradingEnv(
                df=self.test_df,
                pair=self.pair,
                initial_balance=1_000_000,
                sequence_length=5,
                random_start=False
            )
            # Wrap with Monitor for logging
            return Monitor(env)
        
        # Create vectorized environment
        vec_env = DummyVecEnv([make_env])
        
        # Create new VecNormalize wrapper with training statistics
        test_env = VecNormalize(
            vec_env,
            training=False,  # Disable training mode
            norm_obs=vec_normalize.norm_obs,  # Use same normalization settings
            norm_reward=False,  # Disable reward normalization for evaluation
            clip_obs=vec_normalize.clip_obs,
            clip_reward=vec_normalize.clip_reward,
            gamma=vec_normalize.gamma,
            epsilon=vec_normalize.epsilon
        )
        
        # Copy over saved statistics
        test_env.obs_rms = vec_normalize.obs_rms
        test_env.ret_rms = vec_normalize.ret_rms
        
        return test_env
    
    def evaluate_model(
        self,
        n_episodes: int = 1,
        max_steps: Optional[int] = None
    ) -> pd.DataFrame:
        """Run full model evaluation across multiple episodes."""
        all_results = []
        
        for episode in range(n_episodes):
            print(f"\nStarting evaluation episode {episode + 1}/{n_episodes}")
            
            # Run single evaluation episode
            episode_df = self._run_evaluation_episode(max_steps)
            episode_df['episode'] = episode
            all_results.append(episode_df)
            
            # Print episode summary
            # self._print_episode_summary(episode_df)
        
        # Combine all episodes
        combined_df = pd.concat(all_results, ignore_index=True)
        
        # Generate and save analysis
        self._generate_evaluation_report(combined_df)
        
        return combined_df
    
    def _run_evaluation_episode(self, max_steps: Optional[int] = None) -> pd.DataFrame:
        """Execute single evaluation episode and record results."""
        obs = self.env.reset()
        done = False
        
        # Track trades and market data
        trades = []
        steps_data = []
        
        step = 0
        while not done:
            if max_steps and step >= max_steps:
                break
                
            # Get model's action
            action, _ = self.model.predict(obs, deterministic=True)
            
            # Get underlying environment (unwrap from VecEnv)
            # base_env = self.env.envs[0].env.env  # Unwrap Monitor and VecEnv
            base_env = self.env.envs[0].unwrapped  # Unwrap Monitor and VecEnv
            
            # Record pre-step state
            pre_step_price = base_env.current_price
            pre_step_balance = base_env.balance
            
            # Take action in environment
            obs, reward, done, info = self.env.step(action)
            # VecEnv returns numpy arrays - get first element
            reward = reward[0]
            done = done[0]
            info = info[0]
            
            # Record step data
            step_data = {
                'step': step,
                'timestamp': info['timestamp'],
                'net_worth_chg': info['net_worth_chg'],
                'pre_step_price': pre_step_price,
                'post_step_price': info['current_price'],
                'action': Actions(action[0]).name,
                'trading_costs': info['trading_costs'],
                'position_type': info['position_type'],
                'reward': reward,
                'balance': info['balance'],
                'unrealized_pnl': info['unrealized_pnl'],
                'net_worth_chg': info['net_worth_chg'],
                'total_trades': info['total_trades']
            }
            steps_data.append(step_data)
            
            # If trade was closed this step, record trade details
            if info.get('trade_closed', False):
                trade_data = {
                    'entry_time': info['entry_time'],
                    'exit_time': info['exit_time'],
                    'entry_price': info['entry_price'],
                    'exit_price': info['exit_price'],
                    'position_type': info['position_type'],
                    'pnl': info['trade_pnl'],
                    'trade_duration': (info['exit_time'] - info['entry_time']).total_seconds() / 3600
                }
                trades.append(trade_data)
            
            step += 1
        
        # Convert to DataFrame
        steps_df = pd.DataFrame(steps_data)
        
        # Add derived metrics
        if not steps_df.empty:
            steps_df['cumulative_pnl'] = steps_df['net_worth_chg'].cumsum()
            steps_df['drawdown'] = (steps_df['balance'].cummax() - steps_df['balance']) / steps_df['balance'].cummax()
        
        return steps_df
    
    def _print_episode_summary(self, df: pd.DataFrame):
        """Print summary statistics for completed episode."""
        if df.empty:
            print("No episode data to summarize")
            return
            
        print("\nEpisode Summary:")
        print(f"Total Steps: {len(df)}")
        print(f"Initial Balance: ${df['balance'].iloc[0]:,.2f}")
        print(f"Final Balance: ${df['balance'].iloc[-1]:,.2f}")
        print(f"Total Return: {((df['balance'].iloc[-1] / df['balance'].iloc[0]) - 1) * 100:.2f}%")
        print(f"Maximum Drawdown: {df['drawdown'].max():.2%}")
        print(f"Total Trades: {df['total_trades'].iloc[-1]}")
        
    def plot_episode_analysis(self, df: pd.DataFrame, show_trades: bool = True):
        """Generate comprehensive performance visualization."""
        fig, axes = plt.subplots(3, 1, figsize=(15, 12))
        
        # Plot 1: Price and Trades
        ax1 = axes[0]
        ax1.plot(df['timestamp'], df['pre_step_price'], label='Price', color='black', linewidth=1)
        
        if show_trades:
            # Mark trade entries/exits
            long_entries = df[df['action'] == 'LONG'].index
            short_entries = df[df['action'] == 'SHORT'].index
            exits = df[df['action'] == 'NO_POSITION'].index
            
            for idx in long_entries:
                ax1.axvline(x=df['timestamp'].iloc[idx], color='green', alpha=0.2)
            for idx in short_entries:
                ax1.axvline(x=df['timestamp'].iloc[idx], color='red', alpha=0.2)
            for idx in exits:
                ax1.axvline(x=df['timestamp'].iloc[idx], color='black', alpha=0.2)
                
        ax1.set_title('Price and Trading Activity')
        ax1.legend()
        
        # Plot 2: Cumulative PnL
        ax2 = axes[1]
        ax2.plot(df['timestamp'], df['cumulative_pnl'], label='Cumulative PnL', color='blue')
        ax2.set_title('Cumulative Profit/Loss')
        ax2.legend()
        
        # Plot 3: Drawdown
        ax3 = axes[2]
        ax3.fill_between(df['timestamp'], df['drawdown'] * 100, color='red', alpha=0.3)
        ax3.set_title('Drawdown (%)')
        ax3.set_ylim(bottom=0)
        
        plt.tight_layout()
        
        # Save plot
        if self.save_path:
            plt.savefig(self.save_path / 'performance_analysis.png')
        plt.show()
    
    def _generate_evaluation_report(self, df: pd.DataFrame):
        """Generate and save comprehensive evaluation report."""
        if df.empty:
            print("No data available for report generation")
            return
            
        report_path = self.save_path / 'evaluation_report.txt'
        trades_path = self.save_path / 'trades.csv'
        
        # Calculate key metrics
        total_return = ((df['balance'].iloc[-1] / df['balance'].iloc[0]) - 1) * 100
        sharpe_ratio = (df['net_worth_chg'].mean() / df['net_worth_chg'].std()) * np.sqrt(252)  # Annualized
        max_drawdown = df['drawdown'].max() * 100
        
        # Write report
        with open(report_path, 'w') as f:
            f.write("Model Evaluation Report\n")
            f.write("=====================\n\n")
            f.write(f"Testing Period: {df['timestamp'].min()} to {df['timestamp'].max()}\n")
            f.write(f"Total Episodes: {df['episode'].nunique()}\n")
            f.write(f"Total Steps: {len(df)}\n\n")
            
            f.write("Performance Metrics:\n")
            f.write(f"- Total Return: {total_return:.2f}%\n")
            f.write(f"- Sharpe Ratio: {sharpe_ratio:.2f}\n")
            f.write(f"- Maximum Drawdown: {max_drawdown:.2f}%\n")
            f.write(f"- Average Trade PnL: ${df['net_worth_chg'].mean():.2f}\n")
            
            f.write("\nTrading Activity:\n")
            f.write(f"- Total Trades: {df['total_trades'].iloc[-1]}\n")
            
        # Save detailed trade data
        df.to_csv(trades_path, index=False)
        print(f"\nEvaluation report saved to {report_path}")
        print(f"Detailed trade data saved to {trades_path}")

def evaluate_trained_model(
    model: PPO,
    vec_normalize: VecNormalize,
    test_df: pd.DataFrame,
    pair: str,
    save_path: Optional[str] = None
) -> pd.DataFrame:
    """
    Convenience function for running model evaluation.
    
    Args:
        model: Trained PPO model
        vec_normalize: VecNormalize wrapper with saved training statistics
        test_df: Test dataset
        pair: Currency pair being traded
        save_path: Optional path to save evaluation results
        
    Returns:
        DataFrame containing detailed evaluation results
    """
    # Initialize tester
    tester = ModelTester(model, vec_normalize, test_df, pair, save_path)
    
    # Run evaluation
    results_df = tester.evaluate_model(n_episodes=1)
    
    # Generate visualization
    tester.plot_episode_analysis(results_df)
    
    return results_df

In [ ]:
from data_management.dataset_manager import DatasetManager

TICKER = 'AUD_JPY'
dataframe_dir = f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes/{TICKER}_5T_indics_1H_norm.parquet'



df = pd.read_parquet(dataframe_dir)
dataset_manager = DatasetManager()
train_df, val_df, test_df = dataset_manager.split_dataset(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)

# Load your trained model and normalization wrapper
def make_env():
    return ForexTradingEnv(
        df=test_df,  # Your test DataFrame
        pair=TICKER,
        initial_balance=1_000_000.0,
        # trade_size=100_000.0,
        random_start=False
    )

# Create vectorized environment
env = DummyVecEnv([make_env])
env = VecNormalize(
    env,
    norm_obs=True,
    norm_reward=False,  # Disable reward normalization for evaluation

)
# Print current environment's observation space shape
print("New environment observation space shape:", env.observation_space.shape)

model_and_env_path = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes_true_cost/models_and_vecs'


model = PPO.load(f"{model_and_env_path}/{TICKER}_best_model.zip")
vec_normalize = VecNormalize.load(f"{model_and_env_path}/{TICKER}_vec_normalize.pkl", env)
print("Saved environment observation space shape:", vec_normalize.observation_space.shape)


env = DummyVecEnv([make_env])


# Run evaluation
results_df = evaluate_trained_model(
    model=model,
    vec_normalize=vec_normalize,  # Pass the saved normalization wrapper
    test_df=test_df,
    pair=TICKER,
    save_path="model_evaluation"
)

2024-12-02 13:53:17 - oandapyV20.oandapyV20 - INFO - oandapyV20.py:207 - setting up API-client for environment practice
Dataset split sizes:
Training: 1121242 samples (70.0%)
Validation: 240266 samples (15.0%)
Test: 240267 samples (15.0%)
2024-12-02 13:53:17 - ForexEnv2_flat - INFO - forex_env2_flat.py:434 - Selected features for observation space: ['close', 'sma_20', 'sma_50', 'rsi', 'macd', 'macd_signal', 'macd_hist', 'bb_upper', 'bb_middle', 'bb_lower', 'bb_bandwidth', 'bb_percent', 'atr', 'plus_di', 'minus_di', 'adx', 'senkou_span_a', 'senkou_span_b', 'tenkan_sen', 'kijun_sen']
New environment observation space shape: (101,)
Saved environment observation space shape: (101,)
2024-12-02 13:53:18 - ForexEnv2_flat - INFO - forex_env2_flat.py:434 - Selected features for observation space: ['close', 'sma_20', 'sma_50', 'rsi', 'macd', 'macd_signal', 'macd_hist', 'bb_upper', 'bb_middle', 'bb_lower', 'bb_bandwidth', 'bb_percent', 'atr', 'plus_di', 'minus_di', 'adx', 'senkou_span_a', 'senkou